# Bad Photo Generator

### Generating bad photos from good photos

![image info](bad_images.png)

#### Choosing random images from a folder

In [9]:
# loading libraries
import cv2
import numpy as np
import os, random
from PIL import Image

In [10]:
def choose_photos(filename, n_photos=-1):
    if isinstance(filename, set):
        photos = filename
    else:
        photos = set(os.listdir(folder))
    if n_photos == -1:
        return photos
    number_files = len(photos)
    if number_files < n_photos:
        return 'number of photos is larger than the photos in the folder/set'
    chosen_photos = set(random.sample(photos, n_photos))
    photos = photos - chosen_photos
    return photos, chosen_photos

In [67]:
folder = r"D:\Users\Eran\Documents\ITC Alumni Community\‪Side Quest May 2022‬‏\best_photo_selection\data\raw\flickr30k_images"
folder = "test\\synthetic\\bad"
photos = choose_photos(folder)

In [68]:
len(photos)

1000

#### Creating "good" and "bad" folders

In [20]:
try:
    os.mkdir('good') 
except OSError as error: 
    print(error)

In [21]:
try:
    os.mkdir('bad')
except OSError as error: 
    print(error)

#### Copying the random chosen images into the "good" folder

In [7]:
n_good_photos = 7000
photos, good_photos = choose_photos(photos, n_good_photos)

In [8]:
print('Chosen photos:', len(good_photos))
print('Remaining number of photos:', len(photos))
bool(photos & good_photos)

Chosen photos: 7000
Remaining number of photos: 24783


False

In [9]:
import shutil

In [10]:
for photo in good_photos:
    shutil.copy(folder + '\\' + photo, 'good')

#### Generating bad photos

In [28]:
# motion blur filter
#size - in pixels, size of motion blur
#angle - in degrees, direction of motion blur
def apply_motion_blur(image, size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[ (size-1)// 2 , :] = np.ones(size, dtype=np.float32)
    k = cv2.warpAffine(k, cv2.getRotationMatrix2D( (size / 2 -0.5 , size / 2 -0.5 ) , angle, 1.0), (size, size) )
    k = k * (1.0 / np.sum(k))
    return cv2.filter2D(image, -1, k)

In [29]:
# random gaussian blur outer circle filter
def random_blur_out(image, radius=100):
    img_size = (image.shape[0], image.shape[1], 3)
    blurred_img = cv2.GaussianBlur(image, (21, 21), 0)
    mask = np.zeros(img_size, dtype=np.uint8)
    x, y = (random.randint(0, img_size[0]-radius), random.randint(0, img_size[1]-radius))
    mask = cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)
    return np.where(mask==np.array([255, 255, 255]), image, blurred_img)

In [30]:
# random gaussian blur inner circle filter
def random_blur_in(image, radius):
    img_size = (image.shape[0], image.shape[1], 3)
    blurred_img = cv2.GaussianBlur(image, (21, 21), 0)
    mask = np.zeros(img_size, dtype=np.uint8)
    x, y = (random.randint(0, img_size[0]-radius), random.randint(0, img_size[1]-radius))
    mask = cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)
    return np.where(mask==np.array([0, 0, 0]), image, blurred_img)

In [31]:
# illumination filter
def adjust_gamma(image, gamma=0.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)])
    return cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))

In [60]:
# light source filter
def fake_light(image, size=0.5):
    img_size = (image.shape[0], image.shape[1], 3)
    mask = np.zeros(img_size, np.uint8)

    lightsource = cv2.imread('lightsource.png')
    lightsource_size = (int(img_size[0]*size), int(img_size[1]*size))
    lightsource = cv2.resize(lightsource, lightsource_size)
    
    mask = Image.fromarray(np.uint8(mask))
    lightsource = Image.fromarray(np.uint8(lightsource))

    x, y = (random.randint(0, img_size[0]-lightsource_size[0]), random.randint(0, img_size[1]-lightsource_size[1]))

    mask.paste(lightsource, (x, y))
    mask = np.asarray(mask)

    return cv2.addWeighted(img, 1, mask, 1, 0)

In [61]:
def apply_filter(image, filter_name='motion_blur'):
    if filter_name == 'motion_blur':
        return apply_motion_blur(image, 10, 90)
    if filter_name == 'random_blur_out':
        return random_blur_out(image, radius=100)
    if filter_name == 'random_blur_in':
        return random_blur_in(image, radius=100)
    if filter_name == 'bright':
        return adjust_gamma(image, 3)
    if filter_name == 'dark':
        return adjust_gamma(image, 0.3)
    if filter_name == 'light':
        return fake_light(image)
    if filter_name == 'mix':
        return adjust_gamma(apply_motion_blur(fake_light(image), 10, 90), 3)

#### Generating bad photos from bad photos folder

In [69]:
filters = ['motion_blur', 'random_blur_out', 'random_blur_in', 'bright', 'dark', 'light', 'mix']
n_filtered_images = 100

for fil in filters:
    photos, chosen_photos_to_be_filtered = choose_photos(photos, n_filtered_images)
    for img_file in chosen_photos_to_be_filtered:
        # img = cv2.imread(folder + '\\' + img_file)
        img = Image.open(folder + '\\' + img_file).convert('RGB')
        img = np.array(img) 
        img = img[:, :, ::-1].copy() 
        cv2.imwrite('bad\\' + img_file[:-4] + '_' + fil + '.jpg', apply_filter(img , filter_name=fil))
    print('Generated ' + str(len(chosen_photos_to_be_filtered)) + ' for the ' + fil + ' filter')
print('Generated total of ' + str(n_filtered_images*len(filters)) + ' bad images')

Generated 100 for the motion_blur filter
Generated 100 for the random_blur_out filter
Generated 100 for the random_blur_in filter
Generated 100 for the bright filter
Generated 100 for the dark filter
Generated 100 for the light filter
Generated 100 for the mix filter
Generated total of 700 bad images
